## Notebook info

In [ ]:
#!about

In [ ]:
cd $env:temp
del $env:temp\*.xlsx
$PSVersionTable |out-string

## Simple exports 
First examples use `Get-NetAdapter` - not available in all versions of PowerShell, but you can use something else 

**1**  Simplest export - picks some defaults, picks a random file name, opens Excel.

In [ ]:
Get-NetAdapter
Get-NetAdapter | Export-Excel

**2** Specify the file name, and just export some data - **separate operation to open Excel no defaults**

In [ ]:
Get-NetAdapter | Sort-Object  MacAddress |
    Select-Object   Name, MacAddress, Status, LinkSpeed, MediaType |
        Export-excel -path Net1.xlsx

In [ ]:
start .\Net1.xlsx

**3** Make it look a bit nicer  don't treat 802.3 as a number and auto-open Excel

In [ ]:
Get-NetAdapter | Sort-Object  MacAddress |
        Select-Object Name, MacAddress, Status, LinkSpeed, MediaType |
            Export-excel -path Net2.xlsx  `
                -NoNumberConversion MediaType `
                -AutoSize `
                -AutoFilter `
                -BoldTopRow `
                -FreezeTopRow `
                -Show 



**4** Conditional formatting.

When the file is opened you can click the filter option and filter by color

There are multiple ways to do Conditional formatting. This is creates "IF Text Contains "UP" Green foreground" as a rule (with no area limit) and applies it in the export step.  

In [ ]:
$cf = New-ConditionalText -Text "up" -ConditionalTextColor Green -BackgroundColor White -PatternType None
Get-NetAdapter | Sort-Object  MacAddress |
        Select-Object   MacAddress, Status, LinkSpeed, MediaType |
            Export-excel -path Net3.xlsx `
            -Show `
            -AutoSize `
            -AutoFilter `
            -BoldTopRow `
            -FreezeTopRow `
            -ConditionalFormat $cf


## Simpler lists and not starting at A1, and headerless data 

**1** Just send strings - 
Lorem-ipsum is dull - I like  http://www.cupcakeipsum.com  `CupCake-ipsum.txt` is in the download set, just change the path below

If we specify a *start column* we can add data in the blank space on the right

**This will insert 20 columns - one by one**

In [ ]:
del cupcake*.xlsx -ea SilentlyContinue
$cupcake = Get-Content "C:\Users\mcp\Documents\WindowsPowerShell\cupcake-ipsum.txt" -Encoding UTF8
$cupcake.count
$col = 1 ;
foreach ($c in $cupcake) {
        $c -split "\s+" | Export-Excel -Path cupcake1.xlsx -StartColumn $col
        $col++
}

 start cupcake1.xlsx
 
 ###  Make Changes and save as cupcake2 - we'll come back to that.

**DID YOU MAKE AND SAVE CHANGES ?**

**2** Quick Demo of appending rows.

**This will take our 20 columns and append them one after the other **

In [ ]:
$cupcake[0] -split "\s+" | Export-Excel -path cupcakelong.xlsx  -ClearSheet
 foreach ($c in $cupcake) {$c -split "\s+" | Export-Excel -Path  cupcakelong.xlsx -Append   }

 start cupcakelong.xlsx

**3** Taking control of the leaders and/or start / end rows

**This data doesn't have a header row so a normal import won't work correctly**

In [ ]:
Import-Excel -path cupcakelong.xlsx | select -First 3  | ft 


## ERROR EXPECTED !! 
Import-Excel .\cupcake1.xlsx 

We can specify a header row, No headers - which will give properties P1,P2,P3 or we can name the properties

In [ ]:
Import-Excel -HeaderName "String" cupcakelong.xlsx -EndRow 4 | ft 
Import-Excel -NoHeader cupcake1.xlsx    -EndRow 4 | ft

## Sending more data  - tweaking conditional formats

We want to get some data for the next part - use get-sql and query the database of F1 results.
You can get the database from  https://1drv.ms/f/s!AhfYu7-CJv4egbt5FD7Cdxi8jSz3aQ and `installModule GetSQL` but it needs the office ODBC drivers

`Get-Sql -excel -connection "xlfile" ` sets up a SQL connection to an excel file (-Access makes it an access file, -MSSQLServer makes it a sql box, and anything else is an ODBC string)

`-Session xxx` allows multiple connections to be opened at once and defines `xxx` as an alias `for Get-SQL <<session xxx>>` 

Main reason for using `Get-Sql` is we can build up the table as below. RacePos (position) is a string (includes DNF, retired etc.) 


In [ ]:
Get-SQL -Excel -Connection C:\Users\mcp\onedrive\public\f1\f1Results.xlsx -Session f1

**We are doing per row imports there is a faster way to *do* data **  

`-AutoNameRange` means we can then refer to `cells["Wins"]`

This time we will *re-open the file* and apply conditional formatting to the *Wins* column 

In [ ]:
del .\f1winners.xlsx -ea SilentlyContinue

f1 -Table "Results" -GroupBy "DriverName" -Select "DriverName","Count (RaceDate) as wins" -Where "RacePos" -eq "'1'" -Verbose |
    Select-Object -Property drivername,wins | 
        Where-Object -Property wins -ge 5   | 
            Sort-Object wins -Descending    | 
                Export-Excel -path f1winners.xlsx -AutoNameRange  -AutoSize

#re-open excel - we could use -passthru instead. 

$excel = Open-ExcelPackage -Path .\f1winners.xlsx
Add-ConditionalFormatting  -Address $excel.Sheet1.Cells["Wins"] -ThreeIconsSet Flags
Close-ExcelPackage -Show -ExcelPackage $excel

That mapping of values to flags isn't v. good - very few are > 2/3 maximum value or even >  1/3 ... 

So lets take control of which values get which flags - this time we will use `-passthru` to avoid re-opening 
```
        $excel = get | select | sort | Export-Excel -PassThru -Path abc.XLSX <params>
        manipulate $excel
either  Export-Excel -Package $excel   
OR      Close-ExcelPackage    $excel
```

`Close-Excel` supports `-Show` and saves by default: `-NoSave` releases file without updating - worth remembering if you open for read only.

In [ ]:
del .\f1winners.xlsx

$excel = f1 -Table "Results" -GroupBy "DriverName" -Select "DriverName","Count (RaceDate) as Wins" -Where "RacePos" -eq "'1'"  |
    Select-Object -Property Drivername,Wins         | 
        Where-Object -Property Wins -ge 5           |
            Sort-Object -Property Wins -Descending  |
                Export-Excel -path f1winners.xlsx -AutoNameRange  -AutoSize -PassThru

$cf =  Add-ConditionalFormatting  -Address $excel.Sheet1.Cells["Wins"] -ThreeIconsSet Flags  -PassThru

#Examine the icons on the conditional formatting we have set 

$cf            | Out-String
$cf.Icon2 | gm | Out-String

[System.Enum]::GetNames([OfficeOpenXml.ConditionalFormatting.eExcelConditionalFormattingValueObjectType]) | Out-String

#OK ... lets make those percentiles so we get even thirds
$cf.icon2.type="Percentile"
$cf.icon3.type="Percentile"

In [ ]:
Close-ExcelPackage -Show -ExcelPackage $excel

#Leave things tidy
f1 -Close

## Bigger datasets `Send-SQLDataToExcel`,  neater command lines, formulas (named ranges)
A different (and bigger) database - My pictures from Adobe Lightroom
*For the download* I've provided `pictureinfo.csv`  just do `$pictureInfo = import-csv pictureinfo.csv`

First pass is just to see what the data looks like  

In [ ]:
$SQL =@"
SELECT      rootfile.baseName || '.' || rootfile.extension       AS fileName,
            metadata.dateDay         , metadata.dateMonth, metadata.dateYear,
            image.fileFormat         ,
            Image.captureTime       AS dateTaken ,
            metadata.hasGPS          ,
            metadata.FocalLength    AS FocalLength,
            metadata.Aperture       AS apertureValue,
            metadata.ISOSpeedRating AS ISOSpeed,
            metadata.ShutterSpeed   AS shutterSpeedValue,
            Camera.Value            AS cameraModel,
            LensRef.value           AS lensModel
FROM        Adobe_images               image
JOIN        AgLibraryFile              rootFile ON   rootfile.id_local =  image.rootFile
JOIN        AgharvestedExifMetadata    metadata ON      image.id_local =  metadata.image
LEFT JOIN   AgInternedExifLens         LensRef  ON    LensRef.id_Local =  metadata.lensRef
LEFT JOIN   AgInternedExifCameraModel  Camera   ON     Camera.id_local =  metadata.cameraModelRef
WHERE       Camera.Value  like "Pentax%"
ORDER BY    fileName
"@

#Just use the default session this time
Get-SQL -Connection "DSN=LR"
#PowerShell has an implied alias xxx for get-xxx so SQL = Get-SQL ; get-sql takes -SQL parameter and we've put the query in $sql so I run "sequel sequel sequel" 
$pictureInfo = SQL -SQL $sql
Get-SQL -Close

Export-Excel -InputObject $pictureInfo

First revision - exclude some row properties add a Pivot table and chart

In [ ]:
$pictureinfo | Select-Object -Property *,@{n="taken";e={[datetime]$_.datetaken}} |
    Export-Excel -ExcludeProperty datetaken,RowError,RowState,Table,ItemArray,HasErrors `
                 -IncludePivotTable `
                 -PivotRows cameraModel `
                 -PivotColumns lensmodel `
                 -PivotData @{'Taken'='Count'} `
                 -IncludePivotChart `
                 -ChartType ColumnStacked

But the command line is getting unwieldy so use *definitions* like we did for Conditional formatting - they have more options too.    
For a better demo we'll use `Send-SqlDataToExcel` which bolts `Get-SQL` logic onto `Export-Excel` using "Load the whole table in one" instead of pipe row by row. 

In [ ]:
$cDef = New-ExcelChartDefinition -Title "Lens and Camera Usage" `
                                 -ChartType ColumnStacked `
                                 -Row 17 -Column 0 `
                                 -Width 375 -height 500
                                 
$pdef = New-PivotTableDefinition -PivotTableName PicturePivot `
                                 -PivotRows cameraModel       `
                                 -PivotColumns Lensmodel       `
                                 -PivotData  @{'dateTaken'='Count'} `
                                 -PivotChartDefinition $cDef  `
                                 -Activate

$cdef              | out-string
$pdef              | out-string
$pdef.PicturePivot | out-string


`Export-Excel` will take a data table (which is faster than cell by cell). `Send-SqlDataToExcel` wraps getting the table and sending it and takes all the parameters export Excel takes. 

If you have downloaded the csv use the following although it is slower
```
$excel = $pictureinfo | 
    Select-Object -Property *,@{n="taken";e={[datetime]$_.datetaken}} -first 1000 |
         Export-Excel -ExcludeProperty datetaken,RowError,RowState,Table,ItemArray,
                    HasErrors -path picturedemo.xlsx -PivotTableDefinition $pdef -AutoNameRange -PassThru
$sheet = $excel.Sheet1
```


In [ ]:
del picturedemo.xlsx -ea SilentlyContinue
$excel  = Send-SQLDataToExcel -Connection "DSN=LR" -SQL $SQL `
                              -path picturedemo.xlsx `
                              -PivotTableDefinition $pdef `
                              -AutoNameRange `
                              -PassThru
$sheet = $excel.Sheet1


We can access the sheet by name ... and access its cells or ranges of cells in multiple ways

In [ ]:
"Sheet dimension is     $($sheet.Dimension.Start.Address):$($sheet.Dimension.End.Address) "
"Row 2 col 1 has value  $($sheet.cells[2,1].value)"
"Range apertureValue is $($sheet.cells["apertureValue"].Address)"
# Can select with cells["B:B"] cells["2:2"]  cells["a1:b2"]

"Things we can set on cells / ranges"
$sheet.cells["A1"]  |Out-String 


Instead of working on $sheet.cells[<somerange>] we can use `Set-ExcelRange`   
`Set-ExcelColumn` selects a column as a range and calls `Set-ExcelRange` to apply values, number formats, width. etc. If NO column is selected it will add a new column on the right. 

If we want to find a column we can use range names or something like this
```PowerShell
  $col = 1 
  while ($col -lt $sheet.Dimension.Columns -and $sheet.cells[1,$col].Value -ne "apertureValue") {$col++}
  Set-ExcelColumn -Worksheet $sheet -Column $col  -NumberFormat "0.00"
```
But we often need to change more than 1 column  - so the version in the next cell is better.

**The next cell also adds columns containing an Excel formula..**. using auto created ranges in the formula (the data had "apertureValue" and "shutterSpeedValue" as columns, they are now ranges) 
Then 
- Auto fit everything
- Present it as a table, "Light1" style 
- Add save. We could do that with Close-ExcelPackage... but didn't fix the top row do that now -it's an option in Export-Excel so use that and close and show the file in one line


```
$excel  = Send-SQLDataToExcel -PassThru
$sheet  = $excel.Sheet1
Set-ExcelColumn  -Worksheet    $sheet ... # three times
Add-ExcelTable   -Range        $sheet.cells[$($sheet.Dimension.address)] ... 
Export-Excel     -ExcelPackage $excel  ...
```

In [ ]:
1..$sheet.Dimension.Columns | 
    Where-Object {$sheet.cells[1,$_].value -in @("apertureValue","shutterSpeedValue") } |
        Set-ExcelColumn -Worksheet $sheet -NumberFormat "0.00"
        
Set-ExcelColumn -Worksheet $sheet `
                -Heading "f. stop"  `
                -Value "=SQRT(POWER(2,apertureValue))" `
                -NumberFormat '"f/"0.0'

Set-ExcelColumn -Worksheet $sheet `
                -Heading "Exposure time" `
                -Value '=IF(shutterSpeedValue<-1.5,  ROUND(1/POWER(2,shutterSpeedValue),0), IF(shutterSpeedValue<2, 1/POWER(2,shutterSpeedValue),"1/"&TEXT(POWER(2,shutterSpeedValue),"0"   )))' 
                
#this tries to render the cells, which is slow and WINDOWS ONLY
$sheet.Cells[$sheet.Dimension.Address].AutoFitColumns() 

Add-ExcelTable -Range $sheet.cells[$($sheet.Dimension.address)] -TableStyle Light1 -TableName "Pictures"

Export-Excel -ExcelPackage $excel -WorksheetName sheet1 -FreezeTopRow -Show


## More on Formulas 

`-AutoNameRange` with the picture data made each column a named range - apertureValue, cameraModel,dateDay ... ShutterSpeed

If we give the module the value  ` "=SQRT(POWER(2,apertureValue))" `  it puts the formula in the cell (if you change cell values REMOVE THE = SIGN  !)

Excel transforms it to `=SQRT(POWER(2,@apertureValue))`   with an @ sign which means the apertureValue column at this row. So we don't need to use H2,H3,H4

When we export data objects we can make properties hold formulas 

In [ ]:
$data = ConvertFrom-Csv @"
Day,Location,Cucumber,Tomato,Lettuce,Asparagus,Potato
Monday,London,46,35,41,49,30
Tuesday,London,30,26,36,38,27
Wednesday,London,25,26,27,31,30
Thursday,London,47,32,44,21,37
Friday,London,38,40,35,27,39
Saturday,London,32,29,39,32,31
Sunday,London,28,31,37,29,39
Monday,Edinburgh,29,26,36,25,35
Tuesday,Edinburgh,44,48,32,26,28
Wednesday,Edinburgh,46,43,38,41,26
Thursday,Edinburgh,39,39,36,31,20
Friday,Edinburgh,36,38,47,30,24
Saturday,Edinburgh,33,27,39,47,39
Sunday,Edinburgh,43,42,35,37,28
Monday,Glasgow,29,30,25,29,47
Tuesday,Glasgow,32,36,46,38,22
Wednesday,Glasgow,20,33,26,44,27
Thursday,Glasgow,21,29,25,35,46
Friday,Glasgow,39,36,45,28,32
Saturday,Glasgow,22,34,33,33,29
Sunday,Glasgow,27,23,29,24,24
Monday,Birmingham,27,34,49,35,31
Tuesday,Birmingham,39,41,41,31,20
Wednesday,Birmingham,33,46,28,40,47
Thursday,Birmingham,21,46,38,29,20
Friday,Birmingham,31,32,21,32,31
Saturday,Birmingham,44,20,46,26,29
Sunday,Birmingham,47,41,42,23,29
Monday,Cardiff,34,37,40,30,41
Tuesday,Cardiff,47,41,26,24,49
Wednesday,Cardiff,27,40,38,26,32
Thursday,Cardiff,29,28,43,26,22
Friday,Cardiff,21,43,23,37,27
Saturday,Cardiff,42,44,47,38,47
Sunday,Cardiff,37,31,21,28,26
"@

In [ ]:
$data | Add-Member -PassThru -NotePropertyName "Total" -NotePropertyValue "=Cucumber+Tomato+Lettuce+Asparagus+Potato" -Force | export-excel -now -AutoNameRange

In [ ]:
$excelpath          = ".\SubTotals.xlsx"
$WorksheetName      = "Sheet1"
$changeColumnName   = "Day"
$aggregateColumn    =  @{"Cucumber"  = 9
                         "Tomato"    = 9 
                         "Lettuce"   = 9
                         "Asparagus" = 9
                         "Potato"    = 9
                         "Total"     = 9 }   
<# $aggregateFunctions = @{"AVERAGE"=1;"COUNT"=2; "COUNTA"=3;"MAX"=4;"MIN"=5;"PRODUCT";= 6;
                      "STDEV"= 7;"STDEVP"= 8;"SUM"=9;"VAR"= 10;"VARP"=11}  
# CountA=non empty cells VarP and StDevP = "population" not "sample"    #add 100 to ignore hidden cells #>
del $excelpath -ErrorAction SilentlyContinue 

$data               = $data | Sort-Object  $changeColumnName
$Header             = $data[0].PSObject.Properties.Name
$aggregateFormulas        = @{} 

foreach ($k in $aggregateColumn.Keys)  {
    $columnNo       = 0 ; 
    while ($columnNo -lt $header.count  -and $header[$columnNo] -ne $k) {$columnNo ++}
    $aggregateFormulas[$k] =  "=SUBTOTAL({0},{1}{{0}}:{1}{{1}})"  -f $aggregateColumn[$k], (Get-ExcelColumnName ($columnNo+1) ).ColumnName   
}
$aggregateFormulas

In [ ]:
#Work through the data as a we send it to excel adding a subtotals row when the value in .changeColumnName changes. Keep note of the new rows.

$insertedRows       = @()
$previousValue      = $data[0].$changeColumnName
$currentRow         = $lastChangeRow  = 2
$excel              = $data | 
    ForEach-Object -process {
        # if the value in change column has changed output the subtotal row before the data 
        if ($_.$changeColumnName -ne $previousValue) { 
            $newobj = @{$changeColumnName = $previousValue}          
            foreach    ($k in $aggregateFormulas.Keys) {
                $newobj[$k] = $aggregateformulas[$k] -f  $lastChangeRow,  ($currentRow - 1)  # <== Insert subtotal formula for these rows
            }
            [pscustomobject]$newobj
            $insertedRows  += $currentRow 
            $currentRow    += 1 
            $lastChangeRow  = $currentRow 
            $previousValue  = $_.$changeColumnName
        } 
        #Always ouput the data
        $_ 
        $currentRow += 1 
        } -end { #Won't have output subtotals after final rows so put that in
            $newObj = @{$changeColumnName = $previousValue}
            foreach    ($k in $aggregateFormulas.Keys) {
                $newobj[$k] = $aggregateformulas[$k] -f  $lastChangeRow,  ($currentRow - 1)
            }
            [pscustomobject]$newobj
            $insertedRows  += $currentRow 
    }  | Export-Excel  -Path $ExcelPath -PassThru  -AutoSize -AutoFilter -AutoNameRange -BoldTopRow -WorksheetName $WorksheetName -Activate -ClearSheet  

    
#Format the inserted rows. 
$ws                 = $excel.$WorksheetName
foreach ($r in $insertedrows)  {$ws.Row($r).style.font.bold = $true }

$range                 = $ws.Dimension.Address
$ExcelPath             = $excel.File.FullName
$SheetIndex            = $ws.index
Close-ExcelPackage $excel    #Don't show yet


EPPlus can't apply outlining so we'll cheat and use the COM model 

In [ ]:
$excelApp       = New-Object -ComObject "Excel.Application" 
$excelWorkBook  = $excelApp.Workbooks.Open($ExcelPath) 
$ws             = $excelWorkBook.Worksheets.item($SheetIndex)
$null           = $ws.Range($range).Select()
$null           = $excelapp.ActiveCell.AutoOutline()
$null           = $ws.Outline.ShowLevels(1,$null)
$excelWorkBook.Save()
$excelApp.Visible = $true


## Not for this demo :-) 

In [ ]:
ipmo ~\Documents\GitHub\PowerShellPivot\PowerShellPivot.psd1
Get-Item $env:temp\chart.xlsx -ea SilentlyContinue | del 

$locations = $data.location | Sort-Object -Unique

$cd1 = New-ExcelChartDefinition  -ChartType ColumnClustered -XRange Product -YRange $locations -SeriesHeader $locations -Title ("Veg Sales to {0:d}" -f (get-date))
$cd2 = New-ExcelChartDefinition  -ChartType Doughnut        -XRange Product -YRange $locations -SeriesHeader $locations -Title ("Veg Sales to {0:d}" -f (get-date)) -Row 20 

$data | Invoke-PSMelt         -ValueName Sales -VarName Product -Id Day,    Location |
           ConvertTo-CrossTab -ValueName Sales -RowName Product -ColumnName Location -Aggregate Sum   |
                Export-Excel $env:temp\chart.xlsx -ExcelChartDefinition $cd1,$cd2 -AutoNameRange -AutoSize -BoldTopRow -Show

## Comparing and merging 

In [ ]:
compare-worksheet -Referencefile .\cupcake1.xlsx -Differencefile .\cupcake2.xlsx -NoHeader -BackgroundColor LightGreen -FontColor Red -Show
